In [258]:
import ast
import json
import evaluate
import pandas as pd
import numpy as np

from bertopic import BERTopic

In [259]:
df = pd.read_csv("Preprocessed_Data.csv")
df.dropna(inplace=True)
df.drop(columns=["Link"], inplace=True)

In [260]:
df.head(5)

,Headline,Category,Subtitle,Keywords,subtitle_entities,keywords_entities,BERT_topic_num,BERT_topic_prob,BERT_keywords,BERT_kw_topics
0,kaotisk nyttarsnatt ute av kontroll,nyheter,Det har vært en hektisk nyttårsnatt for politi...,nyheter,{},{},-1,0.000000,"['nyttårsnatt', 'politiet', 'hektisk', 'landet...","[22, 120, 89, 109, 159]"
1,fra singel til damemagnet,tema,Ekspertenes beste råd til hvordan du blir attr...,"kjæreste,dating,singel",{},{},12,0.300247,"['datingmarkedet', 'attraktiv', 'råd', 'eksper...","[140, 10, 59, 68, 85]"
2,strom nytt fra januar,tema,"Nye boligregler, bedre jobbpensjon og dyrere f...","økonomi,pensjon,skatt",{},{},-1,0.000000,"['boligregler', 'endringer', 'dyrere', 'bedre'...","[154, 96, 136, 166, 104]"
3,anders kvitt magefettet i superfart 1,tema,"- Hvis folk bare visste hvor enkelt det er, ha...","magefett,vektnedgang,kosthold,styrketrening,",{'Anders Muren': 'PER'},{},98,0.939760,"['magefettet', 'overflødige', 'enkelt', 'raskt...","[98, 79, 18, 80, 137]"
4,taus om overgrepsdom,nyheter,Trump ønsket «henne alt godt» i fjor sommer. N...,"donald trump,ghislaine maxwell,utenriks,nyhete...","{'Trump': 'PER', 'Ghislaine Maxwell': 'PER', '...","{'donald trump': 'PER', 'usa': 'LOC', 'jeffrey...",93,0.785539,"['maxwell', 'trump', 'presidenten', 'unge', 'd...","[8, 93, 7, 32, 54]"


In [261]:
# Drop rows where Category is not a single word
df = df[df.Category.str.contains("-") == False]

In [262]:
def clean(x: str):
    x = x.replace("  ", " ").replace("   ", " ")
    if not x.endswith("."):
        x += "."
    return x

df.Headline = df.Headline.apply(lambda x: 'Overskrift: ' + clean(x))
df.Subtitle = df.Subtitle.apply(lambda x: 'Oppsummering: ' + clean(x))

In [263]:
def extract_entities(x1: str, x2: str):
    # Crappy string replace due to the column-values not being standardized. Also can't replace all "'" with '"' because of the "it's" in the text (and the likes)
    x1 = x1.replace("{'", '{"').replace("'}", '"}').replace("':", '":').replace("',", '",').replace(": '", ': "').replace(", '", ', "')
    x2 = x2.replace("{'", '{"').replace("'}", '"}').replace("':", '":').replace("',", '",').replace(": '", ': "').replace(", '", ', "')
    result = ''
    people = []
    locations = []
    organizations = []
    if (len(x1) > 2):
        string_dict = json.loads(x1)
        # Get keys and values where values are equal to 'PER'
        for key, value in string_dict.items():
            if value == 'PER':
                people.append(key)
            elif value == 'LOC':
                locations.append(key)
            elif value == 'ORG':
                organizations.append(key)
    if (len(x2) > 2):
        string_dict = json.loads(x2)
        # Get keys and values where values are equal to 'PER'
        for key, value in string_dict.items():
            if value == 'PER':
                people.append(key)
            elif value == 'LOC':
                locations.append(key)
            elif value == 'ORG':
                organizations.append(key)
    if len(people) > 0:
        capitalized_people = [person.title() for person in people]
        result += 'Personer: ' + ', '.join(capitalized_people) + '. '
    if len(locations) > 0:
        capitalized_locations = [location.title() for location in locations]
        result += 'Lokasjoner: ' + ', '.join(capitalized_locations) + '. '
    if len(organizations) > 0:
        capitalized_organizations = [organization.title() for organization in organizations]
        result += 'Organisasjoner: ' + ', '.join(capitalized_organizations) + '. '
    return result

df['entities'] = df.apply(lambda x: extract_entities(x['subtitle_entities'], x['keywords_entities']), axis=1)

# Remove subtitle_entities and keywords_entities
df = df.drop(['subtitle_entities', 'keywords_entities'], axis=1)

In [264]:
def extract_keywords(x1: str, x2: str):
    x1 = ast.literal_eval(x1)
    x2 = x2.split(",")
    # Avoid adding the actual answer to the keywords
    if 'nyheter' in x2:
        x2.remove('nyheter')
    if 'Nyheter' in x2:
        x2.remove('Nyheter')
    result = 'Nøkkelord: ' + ', '.join(x1 + x2)
    return result[:-1] + '.'

df['keywords'] = df.apply(lambda x: extract_keywords(x['BERT_keywords'], x['Keywords']), axis=1)
df.drop(['BERT_keywords', 'Keywords'], axis=1, inplace=True)

# Drop BERT_topic_prob as well because we won't use it
df.drop(['BERT_topic_prob'], axis=1, inplace=True)

In [265]:
topic_model = BERTopic()
topic_model = topic_model.load("topicModel")

In [266]:
topic_labels = topic_model.generate_topic_labels()

In [267]:
topic_labels = topic_model.generate_topic_labels()
topic_label_map = {}

for label in topic_labels:
    strings = label.split("_")
    if strings[0] == '-1':
        topic_label_map[strings[0]] = 'ukjent'
    else:
        topic_label_map[strings[0]] = ', '.join(strings[1:])

topic_label_map

{'-1': 'ukjent',
 '0': 'ukraina, russland, russiske',
 '1': 'hun, da, fikk',
 '2': 'drept, mann, politiet',
 '3': 'haaland, erling, braut',
 '4': 'tv, nrk, programmet',
 '5': 'fhi, nakstad, helsedirektør',
 '6': 'therese, johaug, ol',
 '7': 'vedum, trygve, slagsvold',
 '8': 'trump, donald, trumps',
 '9': 'jeg, meg, dette',
 '10': 'kjæresten, paret, kjæreste',
 '11': 'studie, du, ny',
 '12': 'du, sexolog, kvinner',
 '13': 'bok, roman, forfatteren',
 '14': 'omkom, døde, mistet',
 '15': 'veldig, smiler, gladnyhet',
 '16': 'instagram, innlegg, sosiale',
 '17': 'johannes, høsflot, klæbo',
 '18': 'sorg, bort, mistet',
 '19': 'bråthen, clas, brede',
 '20': 'kilo, vekt, ned',
 '21': 'låt, melodi, prix',
 '22': 'politiets, politiet, narkotikapolitiforening',
 '23': 'ingebrigtsen, jakob, warholm',
 '24': 'klima, barth, eide',
 '25': 'bru, tretten, e6',
 '26': 'veronika, stepanova, bolsjunov',
 '27': 'fifa, qatar, vm',
 '28': 'gammel, år, 73',
 '29': 'johny, vassbakk, viggo',
 '30': 'regn, meteor

In [268]:
def extract_topic(x1: str, x2: str):
    x1 = int(x1)
    x2 = ast.literal_eval(x2)
    x2.append(x1)
    result = 'Spådd tema: '
    if -1 in x2:
        x2.remove(-1)
    for i in range(len(x2)):
        if i == len(x2) - 1:
            result += topic_label_map[str(x2[i])]
        else:
            result += topic_label_map[str(x2[i])] + ', '
    return result + '.'

df['topics'] = df.apply(lambda x: extract_topic(x['BERT_topic_num'], x['BERT_kw_topics']), axis=1)    
df.drop(['BERT_topic_num', 'BERT_kw_topics'], axis=1, inplace=True)

In [269]:
df.head()

,Headline,Category,Subtitle,entities,keywords,topics
0,Overskrift: kaotisk nyttarsnatt ute av kontroll.,nyheter,Oppsummering: Det har vært en hektisk nyttårsn...,,"Nøkkelord: nyttårsnatt, politiet, hektisk, lan...","Spådd tema: politiets, politiet, narkotikapoli..."
1,Overskrift: fra singel til damemagnet.,tema,Oppsummering: Ekspertenes beste råd til hvorda...,,"Nøkkelord: datingmarkedet, attraktiv, råd, eks...","Spådd tema: nettbutikker, knallgode, kampanjer..."
2,Overskrift: strom nytt fra januar.,tema,"Oppsummering: Nye boligregler, bedre jobbpensj...",,"Nøkkelord: boligregler, endringer, dyrere, bed...","Spådd tema: selger, leiligheten, huset, økonom..."
3,Overskrift: anders kvitt magefettet i superfar...,tema,Oppsummering: - Hvis folk bare visste hvor enk...,Personer: Anders Muren.,"Nøkkelord: magefettet, overflødige, enkelt, ra...","Spådd tema: du, ekspertene, type, karmøy, døde..."
4,Overskrift: taus om overgrepsdom.,nyheter,Oppsummering: Trump ønsket «henne alt godt» i ...,"Personer: Trump, Ghislaine Maxwell, Jeffrey Ep...","Nøkkelord: maxwell, trump, presidenten, unge, ...","Spådd tema: trump, donald, trumps, trump, ivan..."


In [270]:
df['text'] = df.apply(lambda x: x['Headline'] + ' ' + x['Subtitle'] + ' ' + x['entities'] + ' ' + x['keywords'] + ' ' + x['topics'], axis=1)

In [271]:
for i in range(len(df)):
    if i == 5:
        break
    print(f"Input to model:\n\t{df['text'][i]}\nTARGET:\t{df['Category'][i]}\n")

Input to model:
	Overskrift: kaotisk nyttarsnatt  ute av kontroll. Oppsummering: Det har vært en hektisk nyttårsnatt for politiet flere steder i landet.  Nøkkelord: nyttårsnatt, politiet, hektisk, landet, flere, nyheter. Spådd tema: politiets, politiet, narkotikapolitiforening, julekalender, voksne, kalenderne, julen, julegave, kjære, henlagt, saken, spesialenheten, politiet, prost, kirkeverge.
TARGET:	nyheter

Input to model:
	Overskrift: fra singel til damemagnet. Oppsummering: Ekspertenes beste råd til hvordan du blir attraktiv på datingmarkedet.  Nøkkelord: datingmarkedet, attraktiv, råd, ekspertenes, beste, kjæreste, dating, singel. Spådd tema: nettbutikker, knallgode, kampanjer, kjæresten, paret, kjæreste, sex, lesere, seksualitet, event, shopping, kupp, spennende, kampen, deg, du, sexolog, kvinner.
TARGET:	tema

Input to model:
	Overskrift: strom nytt fra januar. Oppsummering: Nye boligregler, bedre jobbpensjon og dyrere firmabil. Sjekk hva som kommer av endringer fra nyttår.  N

In [272]:
final_df = pd.DataFrame(columns = df.Category.unique())
final_df['text'] = ''
final_df

,nyheter,tema,sport,kjendis,kultur,meninger,annonse,bok,magasinet,okonomi,text


In [273]:
for index, row in df.iterrows():
    final_df.loc[index, row['Category']] = 1
    final_df.loc[index, 'text'] = row['text']

final_df.fillna(0, inplace=True)

In [277]:
# Assert that the sum of columns [nyheter, tema, sport, kjendis, kultur, meninger, annonse, bok, magasinet, okonomi]
# is always equal to 1
assert (final_df.iloc[:, 0:10].sum(axis=1) == 1).all()
# Assert that the df has no NaN values
assert final_df.isnull().values.any() == False

In [274]:
final_df.head(10)

,nyheter,tema,sport,kjendis,kultur,meninger,annonse,bok,magasinet,okonomi,text
0,1,0,0,0,0,0,0,0,0,0,Overskrift: kaotisk nyttarsnatt ute av kontro...
1,0,1,0,0,0,0,0,0,0,0,Overskrift: fra singel til damemagnet. Oppsumm...
2,0,1,0,0,0,0,0,0,0,0,Overskrift: strom nytt fra januar. Oppsummerin...
3,0,1,0,0,0,0,0,0,0,0,Overskrift: anders kvitt magefettet i superfar...
4,1,0,0,0,0,0,0,0,0,0,Overskrift: taus om overgrepsdom. Oppsummering...
5,0,0,1,0,0,0,0,0,0,0,Overskrift: alt var galt. Oppsummering: Hopper...
6,0,0,0,1,0,0,0,0,0,0,Overskrift: setter fyr pa nettet. Oppsummering...
7,0,0,0,0,1,0,0,0,0,0,Overskrift: skadet i kjendis vm. Oppsummering:...
8,1,0,0,0,0,0,0,0,0,0,Overskrift: morketall gitt opp. Oppsummering:...
9,1,0,0,0,0,0,0,0,0,0,Overskrift: thomas skjulte mork hemmelighet i ...


In [276]:
final_df.to_csv('BERT_input.csv', index=False)